### Import Libraries

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.distributions as dist
from torchsummary import summary
import math
import os
import numpy as np
import time
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from pathlib import Path
import re
from skimage.metrics  import structural_similarity as ssim
import plotly.io as pio
import plotly.express as px
import pandas as pd
import pickle
import cv2

pio.renderers.default = 'iframe'

from importlib import reload
import visualization

# locals
import model_architectures

reload(model_architectures)
from model_architectures import VAESegment, Data3DSegToSeg, SegMaskData, Data3DSingleSegToSingleSeg, Data3DSegToSegT1, Data3DSingleSegToSingleSegT1

reload(visualization)
from visualization import brain_diff, viz_slices

### Define Paths

In [7]:
research_dir = r"D:/school/research"
code_dir = os.path.join(research_dir, "code")
p2_dir = os.path.join(code_dir, "paper_two_code")
model_dir = os.path.join(p2_dir, "models")
data_dir = os.path.join(research_dir, "data")
dhcp_rel2 = os.path.join(data_dir, "dhcp_rel2")
processed_dir = os.path.join(dhcp_rel2, "processed")
volume_dir = os.path.join(processed_dir, "volumes")
seg_dir = os.path.join(processed_dir, "segments")
seg_vol_dir = os.path.join(processed_dir, "volume_segments")
pred_dir = os.path.join(dhcp_rel2, "predictions")
seg_pred_dir = os.path.join(pred_dir, "vae_9seg")
metrics_dir = os.path.join(p2_dir, "metrics")
seg_metrics_dir = os.path.join(metrics_dir, "seg_to_seg")

l1_dir = os.path.join(volume_dir, "l1")
l5_dir = os.path.join(volume_dir, "l5")

l1_seg_dir = os.path.join(seg_dir, "l1")
l5_seg_dir = os.path.join(seg_dir, "l5")

l1_seg_vol_dir = os.path.join(seg_vol_dir, "l1")
l5_seg_vol_dir = os.path.join(seg_vol_dir, "l5")

l1_seg_pred_dir = os.path.join(seg_pred_dir, "l1")
l5_seg_pred_dir = os.path.join(seg_pred_dir, "l5")

### Helpers

In [11]:
segments = [
    "Cerebrospinal Fluid",
    "Cortical Grey Matter",
    "White Matter",
    "Background",
    "Ventricle",
    "Cerebelum",
    "Deep Grey Matter",
    "Brainstem",
    "Hippocampus"
]

std_bands = [
    {
        "range": "-inf to -4",
        "low": -np.inf,
        "high": -4
    },
    {
        "range": "-4 to -3",
        "low": -4,
        "high": -3
    },
    {
        "range": "-3 to -2",
        "low": -3,
        "high": -2
    },
    {
        "range": "2 to 3",
        "low": 2,
        "high": 3
    },
    {
        "range": "3 to 4",
        "low": 3,
        "high": 4
    },
    {
        "range": "4 to inf",
        "low": 4,
        "high": np.inf
    }
]

def connected_components_2d(diff):
    _, labels = cv2.connectedComponents(diff)
    
    return labels


def calculate_clusters(diff, threshold=2):
    cluster_diff = diff.copy()
    filter_mask = (diff > -threshold) & (diff < threshold)
    cluster_diff[filter_mask] = 0
    cluster_diff[cluster_diff != 0] = 1
    prepared_diff = cluster_diff.astype('uint8')
    sizes = []
    
    for idx in range(0, 256):
        clusters = connected_components_2d(prepared_diff[:,:,idx])
        values, counts = np.unique(clusters, return_counts=True)
        sizes.extend(counts[1:])
        
    return sizes


def get_overall_metrics(model, data):
    criterion = nn.MSELoss()
    losses = []
    segment_losses = {s: [] for s in segments}
    clusters = []
    segment_clusters = {s: [] for s in segments}
    banded_metrics = {}
    for band in std_bands:
        banded_metrics[f"{band['range']} pixels"] = []
        banded_metrics[f"{band['range']} clusters"] = []
    
    banded_segment_metrics = {}
    for band in std_bands:
        banded_segment_metrics[f"{band['range']} pixels"] = {s: [] for s in segments}
        banded_segment_metrics[f"{band['range']} clusters"] = {s: [] for s in segments}
        
    counter = 0
    for sample in data:
        x = torch.Tensor(sample).reshape((1,) + sample.shape).cuda()
        pred = model(x)
        
        losses.append(float(criterion(x, pred).cpu()))
        
        for idx, segment in enumerate(segments):
            segment_losses[segment].append(float(criterion(x[:,idx,:,:,:], pred[:,idx,:,:,:]).cpu()))
        
        # Get overall cluster data
        og = sample
        pred = pred.reshape(og.shape).detach().cpu().numpy()
        
        diff = np.sum(og, axis=0) - np.sum(pred, axis=0)
        diff_norm = diff / 0.1
        
        clusters.append(np.mean(calculate_clusters(diff_norm)))
        
        # Get segmented cluster data
        for idx, segment in enumerate(segments):
            diff = og[idx,:,:,:] - pred[idx,:,:,:]
            diff_norm = diff / 0.1
            segment_clusters[segment].append(np.mean(calculate_clusters(diff_norm)))
        
        # For overall banded metrics
        for band in std_bands:
            diff = np.sum(og, axis=0) - np.sum(pred, axis=0)
            diff_norm = diff / 0.1
            filter_mask = (diff_norm > band["low"]) & (diff_norm < band["high"])
            banded_diff = diff_norm.copy()
            banded_diff[np.invert(filter_mask)] = 0
            banded_diff[banded_diff != 0] = 1
            banded_diff = banded_diff.astype('uint8')
            
            banded_metrics[f"{band['range']} pixels"].append(np.sum(banded_diff))
            
            sizes = []
            for idx in range(0, 256):
                cc = connected_components_2d(banded_diff[:,:,idx])
                values, counts = np.unique(cc, return_counts=True)
                sizes.extend(counts[1:])
            banded_metrics[f"{band['range']} clusters"].append(np.mean(sizes))
        
        # For segmented banded metrics
        for band in std_bands:
            for idx, segment in enumerate(segments):
                diff = og[idx,:,:,:] - pred[idx,:,:,:]
                diff_norm = diff / 0.1
                
                filter_mask = (diff_norm > band["low"]) & (diff_norm < band["high"])
                banded_diff = diff_norm.copy()
                banded_diff[np.invert(filter_mask)] = 0
                banded_diff[banded_diff != 0] = 1
                banded_diff = banded_diff.astype('uint8')
                
                banded_segment_metrics[f"{band['range']} pixels"][segment].append(np.sum(banded_diff))
                
                sizes = []
                for idx in range(0, 256):
                    cc = connected_components_2d(banded_diff[:,:,idx])
                    values, counts = np.unique(cc, return_counts=True)
                    sizes.extend(counts[1:])
                if len(sizes) == 0:
                    banded_segment_metrics[f"{band['range']} clusters"][segment].append(0)
                else:
                    banded_segment_metrics[f"{band['range']} clusters"][segment].append(np.mean(sizes))

    return losses, segment_losses, clusters, segment_clusters, banded_metrics, banded_segment_metrics


def save_values(losses, segment_losses, clusters, segment_clusters, banded_metrics, banded_segment_metrics, name, run):
    prefix = f"{name}_{run}"
    np.save(os.path.join(seg_metrics_dir, f"{prefix}_losses.npy"), losses)
    np.save(os.path.join(seg_metrics_dir, f"{prefix}_segment_losses.npy"), segment_losses)
    np.save(os.path.join(seg_metrics_dir, f"{prefix}_clusters.npy"), clusters)
    np.save(os.path.join(seg_metrics_dir, f"{prefix}_segment_clusters.npy"), segment_clusters)
    with open(os.path.join(seg_metrics_dir, f"{prefix}_banded_metrics.pkl"), "wb") as f:
        pickle.dump(banded_metrics,f)
    with open(os.path.join(seg_metrics_dir, f"{prefix}_banded_segment_metrics.pkl"), "wb") as f:
        pickle.dump(banded_segment_metrics,f)


def get_single_seg_metrics(model, data):
    criterion = nn.MSELoss()
    losses = []
    clusters = []
    banded_metrics = {}
    for band in std_bands:
        banded_metrics[f"{band['range']} pixels"] = []
        banded_metrics[f"{band['range']} clusters"] = []

    for sample in data:
        x = torch.Tensor(sample).reshape((1,) + sample.shape).cuda()
        pred = model(x)
        
        losses.append(float(criterion(x, pred).cpu()))
        
        og = sample
        pred = pred.reshape(og.shape).detach().cpu().numpy()
        
        diff = np.sum(og, axis=0) - np.sum(pred, axis=0)
        diff_norm = diff / 0.1
        
        clusters.append(np.mean(calculate_clusters(diff_norm)))
        
        for band in std_bands:
            filter_mask = (diff_norm > band["low"]) & (diff_norm < band["high"])
            banded_diff = diff_norm.copy()
            banded_diff[np.invert(filter_mask)] = 0
            banded_diff[banded_diff != 0] = 1
            banded_diff = banded_diff.astype('uint8')
            
            banded_metrics[f"{band['range']} pixels"].append(np.sum(banded_diff))
            
            sizes = []
            for idx in range(0, 256):
                cc = connected_components_2d(banded_diff[:,:,idx])
                values, counts = np.unique(cc, return_counts=True)
                sizes.extend(counts[1:])
            banded_metrics[f"{band['range']} clusters"].append(np.mean(sizes))

    return losses, clusters, banded_metrics

def save_values(losses, segment_losses, clusters, segment_clusters, banded_metrics, banded_segment_metrics, name, run):
    prefix = f"{name}_{run}"
    np.save(os.path.join(seg_metrics_dir, f"{prefix}_losses.npy"), losses)
    np.save(os.path.join(seg_metrics_dir, f"{prefix}_segment_losses.npy"), segment_losses)
    np.save(os.path.join(seg_metrics_dir, f"{prefix}_clusters.npy"), clusters)
    np.save(os.path.join(seg_metrics_dir, f"{prefix}_segment_clusters.npy"), segment_clusters)
    with open(os.path.join(seg_metrics_dir, f"{prefix}_banded_metrics.pkl"), "wb") as f:
        pickle.dump(banded_metrics,f)
    with open(os.path.join(seg_metrics_dir, f"{prefix}_banded_segment_metrics.pkl"), "wb") as f:
        pickle.dump(banded_segment_metrics,f)

        
def save_seg_values(losses, clusters, banded_metrics, name, run):
    prefix = f"{name}_{run}"
    np.save(os.path.join(seg_metrics_dir, f"{prefix}_losses.npy"), losses)
    np.save(os.path.join(seg_metrics_dir, f"{prefix}_clusters.npy"), clusters)
    with open(os.path.join(seg_metrics_dir, f"{prefix}_banded_metrics.pkl"), "wb") as f:
        pickle.dump(banded_metrics,f)
    
def get_overall_metrics_single_seg(base_model_name, data):
    criterion = nn.MSELoss()
    losses = []
    clusters = []
    banded_metrics = {}
    for band in std_bands:
        banded_metrics[f"{band['range']} pixels"] = []
        banded_metrics[f"{band['range']} clusters"] = []
        
    for sample in data:
        pred_img = np.empty_like(sample)
        for segment_number in range(0, len(segments)):
            model_path = os.path.join(model_dir, f"{base_model_name}{segment_number}.pt")
            model = VAESegment(1, 1)
            model.load_state_dict(torch.load(model_path))
            model.cuda()
            model.eval()

            x = torch.Tensor(sample[segment_number]).reshape((1, 1, 256, 256, 256)).cuda()
            pred = model(x)
            pred_img[segment_number] = pred.reshape((256, 256, 256)).detach().cpu().numpy()

        losses.append(float(criterion(torch.Tensor(sample), torch.Tensor(pred_img))))

        diff = np.sum(sample, axis=0) - np.sum(pred_img, axis=0)
        diff_norm = diff / 0.1

        clusters.append(np.mean(calculate_clusters(diff_norm)))

        # For overall banded metrics
        for band in std_bands:
            diff = np.sum(sample, axis=0) - np.sum(pred_img, axis=0)
            diff_norm = diff / 0.1
            filter_mask = (diff_norm > band["low"]) & (diff_norm < band["high"])
            banded_diff = diff_norm.copy()
            banded_diff[np.invert(filter_mask)] = 0
            banded_diff[banded_diff != 0] = 1
            banded_diff = banded_diff.astype('uint8')

            banded_metrics[f"{band['range']} pixels"].append(np.sum(banded_diff))

            sizes = []
            for idx in range(0, 256):
                cc = connected_components_2d(banded_diff[:,:,idx])
                values, counts = np.unique(cc, return_counts=True)
                sizes.extend(counts[1:])
            banded_metrics[f"{band['range']} clusters"].append(np.mean(sizes))
        
    return losses, clusters, banded_metrics

### Load Data

In [9]:
np.random.seed(42)
num_samples = int(len(os.listdir(l1_dir)) / 2)
samples = np.array([i for i in range(0, num_samples)])
np.random.shuffle(samples)

split_val = int(0.8 * num_samples)
train_indices = samples[0:split_val]
val_indices = samples[split_val:]

num_test = int(len(os.listdir(l5_dir)) / 2)
test_indices = np.array([i for i in range(0, num_test)])

train = Data3DSegToSeg(l1_dir, l1_seg_vol_dir, train_indices)
val = Data3DSegToSeg(l1_dir, l1_seg_vol_dir, val_indices)
test = Data3DSegToSeg(l5_dir, l5_seg_vol_dir, test_indices)

train_segments = SegMaskData(l1_seg_dir, train_indices)
val_segments = SegMaskData(l1_seg_dir, val_indices)
test_segments = SegMaskData(l5_seg_dir, test_indices)

batch_size = 1
train_loader = DataLoader(train, batch_size=batch_size)#, num_workers=1)
val_loader = DataLoader(val, batch_size=batch_size)#, num_workers=1)

### Get T2 9 Seg to 9 Seg Metrics

In [115]:
%%time

model_path = os.path.join(model_dir, "vae_rel2t2_seg9_to_seg9.pt")
model = VAESegment(9, 9)
model.load_state_dict(torch.load(model_path))
model.cuda()
model.eval()

print("Collecting train metrics")
train_losses, train_segment_losses, train_clusters, train_segment_clusters, train_banded_metrics, train_banded_segment_metrics = get_overall_metrics(model, train)
save_values(train_losses, train_segment_losses, train_clusters, train_segment_clusters, train_banded_metrics, train_banded_segment_metrics, "seg9_to_seg9", "train")

print("Collecting val metrics")
val_losses, val_segment_losses, val_clusters, val_segment_clusters, val_banded_metrics, val_banded_segment_metrics = get_overall_metrics(model, val)
save_values(val_losses, val_segment_losses, val_clusters, val_segment_clusters, val_banded_metrics, val_banded_segment_metrics, "seg9_to_seg9", "val")

print("Collecting test metrics")
test_losses, test_segment_losses, test_clusters, test_segment_clusters, test_banded_metrics, test_banded_segment_metrics = get_overall_metrics(model, test)
save_values(test_losses, test_segment_losses, test_clusters, test_segment_clusters, test_banded_metrics, test_banded_segment_metrics, "seg9_to_seg9", "test")

CPU times: total: 1d 12h 49min 35s
Wall time: 1h 53min 35s


### Get T1 9 Seg to 9 Seg Metrics

In [10]:
%%time

model_path = os.path.join(model_dir, "vae_rel2t1_seg9_to_seg9.pt")
model = VAESegment(9, 9)
model.load_state_dict(torch.load(model_path))
model.cuda()
model.eval()

train = Data3DSegToSegT1(l1_dir, l1_seg_vol_dir, train_indices)
val = Data3DSegToSegT1(l1_dir, l1_seg_vol_dir, val_indices)
test = Data3DSegToSegT1(l5_dir, l5_seg_vol_dir, test_indices)

print("Collecting train metrics")
train_losses, train_segment_losses, train_clusters, train_segment_clusters, train_banded_metrics, train_banded_segment_metrics = get_overall_metrics(model, train)
save_values(train_losses, train_segment_losses, train_clusters, train_segment_clusters, train_banded_metrics, train_banded_segment_metrics, "seg9_to_seg9_t1", "train")

print("Collecting val metrics")
val_losses, val_segment_losses, val_clusters, val_segment_clusters, val_banded_metrics, val_banded_segment_metrics = get_overall_metrics(model, val)
save_values(val_losses, val_segment_losses, val_clusters, val_segment_clusters, val_banded_metrics, val_banded_segment_metrics, "seg9_to_seg9_t1", "val")

print("Collecting test metrics")
test_losses, test_segment_losses, test_clusters, test_segment_clusters, test_banded_metrics, test_banded_segment_metrics = get_overall_metrics(model, test)
save_values(test_losses, test_segment_losses, test_clusters, test_segment_clusters, test_banded_metrics, test_banded_segment_metrics, "seg9_to_seg9_t1", "test")

C:\compute\envs\torch\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\compute\envs\torch\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars



CPU times: total: 1d 13h 21s
Wall time: 1h 55min 54s


### Get T2 1 Seg to 1 Seg Metrics

In [140]:
%%time

for segment_number in range(0, len(segments)):
    print(f"Analyze model for segment {segment_number}")
    
    # Load data for segment
    train = Data3DSingleSegToSingleSeg(l1_dir, l1_seg_vol_dir, train_indices, segment=segment_number)
    val = Data3DSingleSegToSingleSeg(l1_dir, l1_seg_vol_dir, val_indices, segment=segment_number)
    test = Data3DSingleSegToSingleSeg(l5_dir, l5_seg_vol_dir, test_indices, segment=segment_number)
    
    # Define output paths now :)
    model_path = os.path.join(model_dir, f"vae_rel2t2_seg_to_seg{segment_number}.pt")
    model = VAESegment(1, 1)
    model.load_state_dict(torch.load(model_path))
    model.cuda()
    model.eval()
    
    print("Train")
    train_losses, train_clusters, train_banded_metrics = get_single_seg_metrics(model, train)
    save_seg_values(train_losses, train_clusters, train_banded_metrics, f"seg{segment_number}", "train")
    
    print("Validation")
    val_losses, val_clusters, val_banded_metrics = get_single_seg_metrics(model, val)
    save_seg_values(val_losses, val_clusters, val_banded_metrics, f"seg{segment_number}", "val")
    
    print("Test")
    test_losses, test_clusters, test_banded_metrics = get_single_seg_metrics(model, test)
    save_seg_values(test_losses, test_clusters, test_banded_metrics, f"seg{segment_number}", "test")

Analyze model for segment 0
Train
Validation
Test
Analyze model for segment 1
Train
Validation
Test
Analyze model for segment 2
Train
Validation
Test
Analyze model for segment 3
Train
Validation
Test
Analyze model for segment 4
Train
Validation
Test
Analyze model for segment 5
Train
Validation
Test
Analyze model for segment 6
Train
Validation
Test
Analyze model for segment 7
Train
Validation
Test
Analyze model for segment 8
Train
Validation
Test
CPU times: total: 1d 9h 44min 32s
Wall time: 2h 1min 17s


### Get T1 1 Seg to 1 Seg Metrics

In [12]:
%%time

for segment_number in range(0, len(segments)):
    print(f"Analyze model for segment {segment_number}")
    
    # Load data for segment
    train = Data3DSingleSegToSingleSegT1(l1_dir, l1_seg_vol_dir, train_indices, segment=segment_number)
    val = Data3DSingleSegToSingleSegT1(l1_dir, l1_seg_vol_dir, val_indices, segment=segment_number)
    test = Data3DSingleSegToSingleSegT1(l5_dir, l5_seg_vol_dir, test_indices, segment=segment_number)
    
    # Define output paths now :)
    model_path = os.path.join(model_dir, f"vae_rel2t1_seg_to_seg{segment_number}.pt")
    model = VAESegment(1, 1)
    model.load_state_dict(torch.load(model_path))
    model.cuda()
    model.eval()
    
    print("Train")
    train_losses, train_clusters, train_banded_metrics = get_single_seg_metrics(model, train)
    save_seg_values(train_losses, train_clusters, train_banded_metrics, f"seg{segment_number}_t1", "train")
    
    print("Validation")
    val_losses, val_clusters, val_banded_metrics = get_single_seg_metrics(model, val)
    save_seg_values(val_losses, val_clusters, val_banded_metrics, f"seg{segment_number}_t1", "val")
    
    print("Test")
    test_losses, test_clusters, test_banded_metrics = get_single_seg_metrics(model, test)
    save_seg_values(test_losses, test_clusters, test_banded_metrics, f"seg{segment_number}_t1", "test")

Analyze model for segment 0
Train
Validation
Test
Analyze model for segment 1
Train
Validation
Test
Analyze model for segment 2
Train
Validation
Test
Analyze model for segment 3
Train
Validation
Test
Analyze model for segment 4
Train
Validation
Test
Analyze model for segment 5
Train
Validation
Test
Analyze model for segment 6
Train
Validation
Test
Analyze model for segment 7
Train
Validation
Test
Analyze model for segment 8
Train
Validation
Test
CPU times: total: 1d 9h 13min 29s
Wall time: 1h 56min 40s


### Get T2 1 Seg to 1 Seg Overall Metrics

In [13]:
%%time

train = Data3DSegToSeg(l1_dir, l1_seg_vol_dir, train_indices)
val = Data3DSegToSeg(l1_dir, l1_seg_vol_dir, val_indices)
test = Data3DSegToSeg(l5_dir, l5_seg_vol_dir, test_indices)

print("Getting train metrics")
train_losses, train_clusters, train_banded_metrics = get_overall_metrics_single_seg("vae_rel2t2_seg_to_seg", train)
save_seg_values(train_losses, train_clusters, train_banded_metrics, "overall_single_seg_t2", "train")
    
print("Validation")
val_losses, val_clusters, val_banded_metrics = get_overall_metrics_single_seg("vae_rel2t2_seg_to_seg", val)
save_seg_values(val_losses, val_clusters, val_banded_metrics, "overall_single_seg_t2", "val")

print("Test")
test_losses, test_clusters, test_banded_metrics = get_overall_metrics_single_seg("vae_rel2t2_seg_to_seg", test)
save_seg_values(test_losses, test_clusters, test_banded_metrics, "overall_single_seg_t2", "test")

Getting train metrics
Validation
Test
CPU times: total: 4h 22min 22s
Wall time: 27min 58s


### Get T1 1 Seg to 1 Seg Overall Metrics

In [14]:
%%time

train = Data3DSegToSegT1(l1_dir, l1_seg_vol_dir, train_indices)
val = Data3DSegToSegT1(l1_dir, l1_seg_vol_dir, val_indices)
test = Data3DSegToSegT1(l5_dir, l5_seg_vol_dir, test_indices)

print("Getting train metrics")
train_losses, train_clusters, train_banded_metrics = get_overall_metrics_single_seg("vae_rel2t1_seg_to_seg", train)
save_seg_values(train_losses, train_clusters, train_banded_metrics, "overall_single_seg_t1", "train")
    
print("Validation")
val_losses, val_clusters, val_banded_metrics = get_overall_metrics_single_seg("vae_rel2t1_seg_to_seg", val)
save_seg_values(val_losses, val_clusters, val_banded_metrics, "overall_single_seg_t1", "val")

print("Test")
test_losses, test_clusters, test_banded_metrics = get_overall_metrics_single_seg("vae_rel2t1_seg_to_seg", test)
save_seg_values(test_losses, test_clusters, test_banded_metrics, "overall_single_seg_t1", "test")

Getting train metrics
Validation
Test
CPU times: total: 4h 22min 30s
Wall time: 28min 10s
